# Link to the datasets

https://www.kaggle.com/datasets/arturdragunov/uk-weekly-real-estate-listings-2022-2023?select=UK_2023-01-15.csv

https://www.kaggle.com/datasets/arturdragunov/france-weekly-real-estate-listings-2022-2023

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [32]:
uk = pd.read_csv('UK_clean_unique.csv')

In [33]:
uk.columns

Index(['MYYYY_generation', 'is_virtual_tour', 'is_audio_tour', 'county',
       'date_of_data_generation', 'is_premium_listing', 'latitude',
       'publication_id', 'longitude', 'isNew', 'num_bathrooms.x',
       'num_bedrooms.x', 'num_floors', 'num_sofa', 'price', 'price_direction',
       'property_badge', 'property_type', 'country', 'lagged_CPI',
       'crime_rate', 'bank_rate', 'lagged_unemployment',
       'lagged_arrears_over_2.5', 'volatility_value',
       'num_bedrooms_categorical', 'num_bathrooms_categorical', 'nr_weeks'],
      dtype='object')

In [34]:
fr = pd.read_csv('France_clean_unique.csv')

In [35]:
usa = pd.read_csv('housing.csv')

In [36]:
usa.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'type', 'sqfeet', 'beds',
       'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'laundry_options', 'parking_options', 'image_url', 'description', 'lat',
       'long', 'state'],
      dtype='object')

In [37]:
def calculate_original_value(x):
    """
    Calculate the original value from a variable x which is 1.1% of the original.
    """
    return x / 0.011

In [38]:
usa['orig_price'] = usa['price'].apply(calculate_original_value)

In [40]:
median_price_france = fr['price'].median()
median_price_uk = uk['price'].median()
median_price__usa = usa['orig_price'].mean()
median_price_france, median_price_uk, median_price__usa


(225750.0, 255000.0, 802338.3925887429)

In [41]:
down_payment_fraction = 0.20
loan_term_years = 25
loan_term_months = loan_term_years * 12

def calculate_mortgage(country_name, property_price, annual_interest_rate, annual_net_salary):
    """
    Calculate monthly mortgage payment and years to repay using given parameters.
    """
    monthly_interest_rate = annual_interest_rate / 12

    down_payment = property_price * down_payment_fraction
    loan_amount = property_price - down_payment

    monthly_payment = (
        loan_amount * monthly_interest_rate * (1 + monthly_interest_rate) ** loan_term_months
    ) / ((1 + monthly_interest_rate) ** loan_term_months - 1)

    years_to_repay = (monthly_payment * loan_term_months) / annual_net_salary

    return {
        "Country": country_name,
        "Property Price": property_price,
        "Down Payment": down_payment,
        "Loan Amount": loan_amount,
        "Monthly Payment": monthly_payment,
        "Years to Repay": years_to_repay
    }

france_data = {
    "country_name": "France",
    "property_price": median_price_france,  
    "annual_interest_rate": 0.0349,  
    "annual_net_salary": 43994  
}

uk_data = {
    "country_name": "United Kingdom",
    "property_price": median_price_uk,  
    "annual_interest_rate": 0.048,  
    "annual_net_salary": 40000 
}

usa_data = {
    "country_name": "USA",
    "property_price": median_price__usa,  
    "annual_interest_rate": 0.07,  
    "annual_net_salary": 157_000 
}

france_results = calculate_mortgage(**france_data)
uk_results = calculate_mortgage(**uk_data)
usa_results = calculate_mortgage(**usa_data)


def display_results(results):
    print(f"Country: {results['Country']}")
    print(f"Property Price: {results['Property Price']:.2f}")
    print(f"Down Payment: {results['Down Payment']:.2f}")
    print(f"Loan Amount: {results['Loan Amount']:.2f}")
    print(f"Monthly Payment: {results['Monthly Payment']:.2f}")
    print(f"Years to Repay: {results['Years to Repay']:.2f}\n")

display_results(france_results)
display_results(uk_results)
display_results(usa_results)


Country: France
Property Price: 225750.00
Down Payment: 45150.00
Loan Amount: 180600.00
Monthly Payment: 903.16
Years to Repay: 6.16

Country: United Kingdom
Property Price: 255000.00
Down Payment: 51000.00
Loan Amount: 204000.00
Monthly Payment: 1168.91
Years to Repay: 8.77

Country: USA
Property Price: 802338.39
Down Payment: 160467.68
Loan Amount: 641870.71
Monthly Payment: 4536.61
Years to Repay: 8.67



In [42]:
def sqft_to_m2(sqft):
    m2 = sqft * 0.092903
    return m2

In [49]:
tolerance = 0.1

france_close_to_avg = fr[
    (fr['price'] >= (median_price_france * (1 - tolerance))) &
    (fr['price'] <= (median_price_france * (1 + tolerance)))
]

france_summary = france_close_to_avg[['price', 'livingArea', 'num_rooms.x', 'City', 'isNew', 'isExclusiveness']].head()

uk_close_to_avg = uk[
    (uk['price'] >= (median_price_uk * (1 - tolerance))) &
    (uk['price'] <= (median_price_uk * (1 + tolerance)))
]

uk_summary = uk_close_to_avg[['price', 'num_bedrooms_categorical', 'num_bathrooms_categorical', 'county', 'isNew']].head()

usa_close_to_avg = usa[
    (usa['orig_price'] >= (median_price__usa * (1 - tolerance))) &
    (usa['orig_price'] <= (median_price__usa * (1 + tolerance)))
]

usa_summary = usa_close_to_avg[['orig_price', 'sqfeet', 'beds', 'baths', 'region', 'type']].head()
usa_summary['m2'] = usa_summary['sqfeet'].apply(sqft_to_m2)
usa_summary = usa_summary.drop(columns=['sqfeet'])
usa_summary = usa_summary[['orig_price', 'm2','beds', 'baths', 'region', 'type']]
france_summary


,price,livingArea,num_rooms.x,City,isNew,isExclusiveness
20,225000,36.00,2,Marseille,False,True
35,210000,42.00,2,Lyon,False,True
44,215000,40.00,2,Toulouse,False,True
50,234000,64.00,3,Toulouse,False,False
55,239000,69.29,3,Toulouse,False,True


In [50]:
uk_summary

,price,num_bedrooms_categorical,num_bathrooms_categorical,county,isNew
0,250000,small,medium,London,False
28,270000,medium,small,West Midlands,False
31,260000,medium,medium,West Midlands,False
47,250000,medium,medium,West Midlands,True
55,239995,small,medium,West Midlands,True


In [51]:
usa_summary

,orig_price,m2,beds,baths,region,type
5082,722727.272727,371.612000,4,4.0,san diego,house
5894,768181.818182,160.071869,2,2.0,san diego,apartment
5898,768181.818182,160.071869,2,2.0,san diego,apartment
6427,772727.272727,353.031400,5,4.0,santa barbara,house
6512,722727.272727,248.887137,5,3.0,santa barbara,house
